# Modélisation

### Import des modules et du fichier nettoyé

In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from projet_3_struct.funcs import (
    Multi_usage,
    Categorie_anne_construction,
    Categorie_nb_etage,
    Categorie_nb_batiment,
    calcul_part_habitation,
)

from scipy import stats

#Selection
from sklearn.model_selection import (
    train_test_split,
    GridSearchCV, 
    cross_validate,
)
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error ,mean_absolute_percentage_error, root_mean_squared_error
from sklearn.inspection import permutation_importance

#Preprocess
from sklearn.compose import ColumnTransformer
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

#Modèles
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

In [67]:
building_consumption = pd.read_csv("building_consumption_pour_analyses.csv")

del building_consumption["Conso_elec_mesure"]

In [68]:
target = "SiteEnergyUseWN(kBtu)_ajuste"

# les variables prédictives

X = building_consumption.drop(columns=[target])

# la variable cible

y = building_consumption[target]

X.head()

,PropertyGFATotal,PropertyGFAParking,Conso_gaz_mesure,Emission_steam_mesure,Groupe_anne_construction_label,Groupe_nb_etages_label,Groupe_nb_batiments_label,BuildingType_Multifamily HR (10+),BuildingType_Multifamily LR (1-4),BuildingType_Multifamily MR (5-9),...,BuildingType_SPS-District K-12,CouncilDistrictCode_1,CouncilDistrictCode_2,CouncilDistrictCode_3,CouncilDistrictCode_4,CouncilDistrictCode_5,CouncilDistrictCode_6,CouncilDistrictCode_7,Usage_multiple_Mono usage,Usage_multiple_Multi usage
0,88434,0,1,1,0,2,0,False,False,False,...,False,False,False,False,False,False,False,True,True,False
1,103566,15064,1,0,1,2,0,False,False,False,...,False,False,False,False,False,False,False,True,False,True
2,61320,0,1,1,0,2,0,False,False,False,...,False,False,False,False,False,False,False,True,True,False
3,175580,62000,1,0,1,2,0,False,False,False,...,False,False,False,False,False,False,False,True,False,True
4,97288,37198,1,0,1,0,0,False,False,False,...,False,False,False,False,False,False,False,True,True,False


### Comparaison de différents modèles supervisés

A réaliser :
* Pour chaque algorithme que vous allez tester, vous devez :
    * Réaliser au préalable une séparation en jeu d'apprentissage et jeu de test via une validation croisée.
    * Si les features quantitatives que vous souhaitez utiliser ont des ordres de grandeur très différents les uns des autres, et que vous utilisez un algorithme de regression qui est sensible à cette différence, alors il faut réaliser un scaling (normalisation) de la donnée au préalable.
    * Entrainer le modèle sur le jeu de Train
    * Prédire la cible sur la donnée de test (nous appelons cette étape, l'inférence).
    * Calculer les métriques de performance R2, MAE et RMSE sur le jeu de train et de test.
    * Interpréter les résultats pour juger de la fiabilité de l'algorithme.
* Vous pouvez choisir par exemple de tester un modèle linéaire, un modèle à base d'arbres et un modèle de type SVM
* Déterminer le modèle le plus performant parmi ceux testés.

In [69]:
# CODE COMPARAISON DES MODELES
#Regression linéaire

reg = LinearRegression()
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

reg.fit(X_train, y_train)

y_pred_Reg = reg.predict(X_test)

print(f"R2: {reg.score(X_test, y_test)}")
print(f"RMSE: {root_mean_squared_error(y_test, y_pred_Reg)}")
print(f"MAE: {mean_absolute_error(y_test, y_pred_Reg)}")

R2: 0.4864252378601245
RMSE: 5654362.234140306
MAE: 3215400.5111903152


In [ ]:
dummy_regr = DummyRegressor(strategy="mean")

dummy_regr.fit(X_train, y_train)

y_pred_Dummy = dummy_regr.predict(X_test)

print(f"R2 dummy: {dummy_regr.score(X_test, y_test)}")
print(f"RMSE dummy: {root_mean_squared_error(y_test, y_pred_Dummy)}")
print(f"MAE dummy: {mean_absolute_error(y_test, y_pred_Dummy)}")

R2 dummy: -7.034892114865521e-05
RMSE: 7890364.494383279
MAE: 4521846.847475139


### Optimisation et interprétation du modèle

A réaliser :
* Reprennez le meilleur algorithme que vous avez sécurisé via l'étape précédente, et réalisez une GridSearch de petite taille sur au moins 3 hyperparamètres.
* Si le meilleur modèle fait partie de la famille des modèles à arbres (RandomForest, GradientBoosting) alors utilisez la fonctionnalité feature importance pour identifier les features les plus impactantes sur la performance du modèle. Sinon, utilisez la méthode Permutation Importance de sklearn.